In [1]:
from utils import download_all_models, suppress_stdout_stderr

TOKEN = ""
download_all_models()

/home/ubuntu/mykernel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from maianet import MaiaNet
from soyatrans import SoyaTrans
from build import build_model
from convnext import Autoencoder
from coreplant import Classifier

model_objects = [SoyaTrans, MaiaNet, build_model, Autoencoder, Classifier]
model_names = ["soyatrans","maianet", "tswinf", "convnext", "coreplant"]

datasets = ["nirmal", "pungliya", "mendeley"]
dataset_num_classes = [5, 3, 11]

model_files = [f"models/{model}_{dataset}.pth" for model in model_names for dataset in datasets]
model_instances = [model(num_classes = num_classes) for model in model_objects for num_classes in dataset_num_classes]

model_files

['models/soyatrans_nirmal.pth',
 'models/soyatrans_pungliya.pth',
 'models/soyatrans_mendeley.pth',
 'models/maianet_nirmal.pth',
 'models/maianet_pungliya.pth',
 'models/maianet_mendeley.pth',
 'models/tswinf_nirmal.pth',
 'models/tswinf_pungliya.pth',
 'models/tswinf_mendeley.pth',
 'models/convnext_nirmal.pth',
 'models/convnext_pungliya.pth',
 'models/convnext_mendeley.pth',
 'models/coreplant_nirmal.pth',
 'models/coreplant_pungliya.pth',
 'models/coreplant_mendeley.pth']

In [3]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

vdata = datasets.ImageFolder("vizdata/Visualization/pungliyavithika", transform=transform)
ndata = datasets.ImageFolder("vizdata/Visualization/nirmalsankana", transform=transform)
mdata = datasets.ImageFolder("vizdata/Visualization/mendeley", transform=transform)

def match_dataset(model_name):
    if "nirmal" in model_name:
        dataset = ndata
        dataset_name = 'nirmal'
    elif "mendeley" in model_name:
        dataset = mdata
        dataset_name = 'mendeley'
    else:
        dataset = vdata
        dataset_name = 'pungliya'
    return dataset, dataset_name


In [4]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TSwinModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        out = self.model(x)
        return out["sup"]  # only return the classification output since it is a Munch object

def load_instance(model, file):
    checkpoint = torch.load(f"{file}")
    model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)
    print(f"{file} loaded")

    if "maianet" in file:
        target_layer = [model.maia_4.conv3[0]]
    elif "soyatrans" in file:
        target_layer = [model.stage1.downsample]
    elif "tswinf" in file:
        target_layer = [model.stage4[-1].attns[0].get_v]
        model = TSwinModel(model)
        # target_layer = [model.LCA.conv1[0]]
    elif 'coreplant' in file:
        target_layer = [model.encoder.model.conv_head]
    elif 'convnext' in file:
        target_layer = [model.encoder.stage4[1]]
    return model, target_layer

models = {file: load_instance(model, file) for file, model in zip(model_files, model_instances)}

models/soyatrans_nirmal.pth loaded
models/soyatrans_pungliya.pth loaded
models/soyatrans_mendeley.pth loaded
models/maianet_nirmal.pth loaded
models/maianet_pungliya.pth loaded
models/maianet_mendeley.pth loaded
models/tswinf_nirmal.pth loaded
models/tswinf_pungliya.pth loaded
models/tswinf_mendeley.pth loaded
models/convnext_nirmal.pth loaded
models/convnext_pungliya.pth loaded
models/convnext_mendeley.pth loaded
models/coreplant_nirmal.pth loaded
models/coreplant_pungliya.pth loaded
models/coreplant_mendeley.pth loaded


In [10]:
import gc

import numpy as np
import torch
from PIL import Image
from pytorch_grad_cam import (
    GradCAM,
    GradCAMPlusPlus,
    ScoreCAM,  # Needed for isinstance check
)
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from tqdm import tqdm


def tensor_to_rgb_image(tensor):
    img = tensor.clone().detach().cpu()
    img = img * 0.5 + 0.5  # reverse normalization
    img = img.permute(1, 2, 0).numpy()
    img = np.clip(img, 0, 1)
    return img
    
def plot(model, image, image_path, class_index, cams, output_dir="output", model_name="default_model"):
    results = {}
    paths = {}

    # Get class folder and image base name
    class_folder = os.path.basename(os.path.dirname(image_path))  # e.g., 'class1'
    base_name = os.path.splitext(os.path.basename(image_path))[0]  # e.g., 'image1'

    # Final output path: output/model_name/class_folder/
    final_output_dir = os.path.join(output_dir, model_name, class_folder)
    os.makedirs(final_output_dir, exist_ok=True)

    device = next(model.parameters()).device
    input_tensor = image.unsqueeze(0).to(device)
    rgb_img = tensor_to_rgb_image(image)
    target = [ClassifierOutputTarget(class_index)]

    # Save original image
    original_path = os.path.join(final_output_dir, f"{base_name}_original.jpg")
    original_image = Image.fromarray((rgb_img * 255).astype(np.uint8))
    original_image.save(original_path)
    results['original'] = original_image
    paths['original'] = original_path

    # Save CAM images
    for name, cam_method in cams.items():
        if isinstance(cam_method, ScoreCAM):
            with torch.no_grad():
                grayscale_cam = cam_method(input_tensor=input_tensor, targets=target)[0]
        else:
            grayscale_cam = cam_method(input_tensor=input_tensor, targets=target)[0]

        cam_image = Image.fromarray(show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True))
        save_path = os.path.join(final_output_dir, f"{base_name}_{name}.jpg")
        cam_image.save(save_path)
        results[name] = cam_image
        paths[name] = save_path

    return results, paths


In [12]:
done_models = []
done_models.append(model_name)

In [16]:
import torch
import os
import io
import gc


all_data = []

for model_name, model_item in models.items():

    model_name = model_name.removeprefix('models/')

    if model_name in done_models:
        continue
    model, target_layers = model_item
    dataset, dataset_name = match_dataset(model_name)
    class_names = dataset.classes
    print(model_name, dataset_name)

    model = model.cuda() if torch.cuda.is_available() else model.cpu()
    model.eval()

    cams = {
        "Grad-CAM": GradCAM(model=model, target_layers=target_layers),
        "Grad-CAM++": GradCAMPlusPlus(model=model, target_layers=target_layers),
        "Score-CAM": ScoreCAM(model=model, target_layers=target_layers),
    }

    with tqdm(total=len(dataset), desc=f"{model_name}", leave=True) as pbar:
        for idx in range(len(dataset)):
            image, label = dataset[idx]
            image_path, _ = dataset.imgs[idx]

            # Suppress plot outputs
            with suppress_stdout_stderr():
            # if True:
                input_tensor = image.unsqueeze(0).to(device)
                output_dir = f"output/{model_name}"

                cam_results, paths = plot(model, image, image_path, label, cams, output_dir="output", model_name =model_name)
                    
                data_entry = {
                    "model": model_name,
                    "dataset": dataset_name,
                    "original": cam_results['original'],
                    "gradcam": cam_results["Grad-CAM"],
                    "gradcam++": cam_results["Grad-CAM++"],
                    "scorecam": cam_results["Score-CAM"],
                    "label": class_names[label],
                    "label_index": label,
                    "paths":paths
                }
                all_data.append(data_entry)
            pbar.update(1)
        
    del model, cams
    torch.cuda.empty_cache()
    gc.collect()
    done_models.append(model_name)

soyatrans_pungliya.pth pungliya


soyatrans_pungliya.pth: 100%|██████████| 45/45 [01:30<00:00,  2.01s/it]


soyatrans_mendeley.pth mendeley


soyatrans_mendeley.pth: 100%|██████████| 165/165 [05:34<00:00,  2.03s/it]


maianet_nirmal.pth nirmal


...

coreplant_nirmal.pth: 100%|██████████| 75/75 [02:11<00:00,  1.75s/it]


coreplant_pungliya.pth pungliya


coreplant_pungliya.pth: 100%|██████████| 45/45 [01:18<00:00,  1.74s/it]


coreplant_mendeley.pth mendeley


coreplant_mendeley.pth: 100%|██████████| 165/165 [04:47<00:00,  1.75s/it]


In [17]:
# import os
# from utils import upload
# models = [f"models/{i}" for i in os.listdir('models') if i.endswith('.pth')]
# upload(models)

from datasets import Dataset, Features, Image as HFImage

columns = {key: [d[key] for d in all_data] for key in all_data[0]}

upload_dataset = Dataset.from_dict(columns)

# Convert columns to Image features after dataset creation
upload_dataset = upload_dataset.cast_column("original", HFImage())
upload_dataset = upload_dataset.cast_column("gradcam", HFImage())
upload_dataset = upload_dataset.cast_column("gradcam++", HFImage())
upload_dataset = upload_dataset.cast_column("scorecam", HFImage())

upload_dataset.push_to_hub("agri_viz", token=TOKEN)

Map: 100%|██████████| 1350/1350 [00:00<00:00, 1481.23 examples/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:12<00:00, 12.68s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/omkar334/agri_viz/commit/64ebb277f715e776f0d17d6acb8700330287f08e', commit_message='Upload dataset', commit_description='', oid='64ebb277f715e776f0d17d6acb8700330287f08e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/omkar334/agri_viz', endpoint='https://huggingface.co', repo_type='dataset', repo_id='omkar334/agri_viz'), pr_revision=None, pr_num=None)

In [20]:
# !zip -r output.zip output

In [25]:
from utils import upload

paths = ['output.zip']
upload(paths)

Uploading files:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading output.zip...



put.zip:   0%|          | 0.00/57.4M [00:00<?, ?B/s]
put.zip:   1%|▏         | 803k/57.4M [00:00<00:07, 7.98MB/s]
put.zip:   4%|▍         | 2.54M/57.4M [00:00<00:04, 13.3MB/s]
put.zip:  28%|██▊       | 16.0M/57.4M [00:00<00:01, 28.8MB/s]
put.zip:  38%|███▊      | 21.7M/57.4M [00:00<00:01, 31.9MB/s]
put.zip:  43%|████▎     | 24.7M/57.4M [00:00<00:01, 29.6MB/s]
put.zip:  56%|█████▌    | 32.0M/57.4M [00:01<00:00, 25.6MB/s]
output.zip: 100%|██████████| 57.4M/57.4M [00:01<00:00, 33.0MB/s]
Uploading files: 100%|██████████| 1/1 [00:02<00:00,  2.72s/it]

✅ Upload complete.
